MARK

In [1]:
# Load data
import pandas as pd 

file_path='assets/datasets/clean/250604-consolidated_survey.xlsx'
df_load=pd.read_excel(file_path).drop(columns=['Unnamed: 0'])

In [42]:
# df_load['inv_sectors'].head()
df_load['inv_time'].head()
# df_load.head()

0    1.0
1    1.0
2    4.0
3    4.0
4    2.0
Name: inv_time, dtype: float64

In [3]:
df_load.columns

Index(['start_date', 'end_date', 'survey_status', 'completion_level',
       'completion_time', 'survey_completed', 'recorded_date', 'response_id',
       'distribution_channel', 'user_language', 'recaptcha_score',
       'gender_male', 'age', 'children_num', 'edu_level', 'edu_field',
       'edu_field_other', 'working_status', 'working_status_other',
       'expertise_area', 'expertise_area_other', 'work_industries',
       'work_industries_other', 'work_company_size', 'roles_cat', 'inv_time',
       'inv_motivation', 'inv_motivation_other', 'ba_group', 'ba_group_name',
       'ba_group_name_other', 'ba_group_reason', 'ba_group_reason_other',
       'inv_num', 'inv_companies_last2y', 'inv_amount_average',
       'inv_capital_stake', 'inv_dd_duration', 'inv_criteria',
       'inv_criteria_other', 'inv_sectors', 'inv_sectors_other', 'inv_process',
       'inv_process_other', 'inv_visits', 'non_fin_contr',
       'non_fin_contr_other', 'child_bias', 'inv_goal', 'inv_exit',
       'inv_re

In [5]:
# Constants

COLS={
    'inv_time': ['Investor Experience', ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],],  
    'inv_motivations': ['Motivations', [
        # ['Support innovative startups or entrepreneurs', 'Potential high returns on investments', 'To diversify my investment portfolio', 'Recommendation from friends', 'Benefits from tax breaks', 'My financial capacity', 'My family background', 'Other' ],
        # ['Desire to support innovative startups or entrepreneurs', 'Potential high returns on investments', 'Diversifying my investment portfolio', 'Recommendation from friends or media influence', 'Benefits from tax breaks', 'Business reasons (you could be a supplier, etc.)', 'Other' ],
        ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Financial capacity', 'Family background', 'Other' ],
        ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Business reasons', 'Other' ],
    ],], # Not same labels
    'ba_group': ['Part of BA Network', ['Yes', 'No'],], # Boolean Only BA
    'ba_group_name': ['BA Network'], # Only BA
    'ba_group_reason': ['Part of BA Network Motivations',
        # ['To gain access to a larger number of investment opportunities', 'To invest in mutual funds set up by the network', 'To leverage the experience of other network members', 'To benefit from educational or training sessions', 'To expand my personal and professional network', 'To coinvest with other investors', 'Other',],
        ['More opportunities', 'Network funds', "Members' experience", 'Training', 'Expand network', 'Coinvest', 'Other',],
    ], # Only BA  
    'inv_num': ['Number of Investments', ['None', '1-2', '3-5', '6-10', '11-20', '>20'],],
    'inv_companies_last2y': ['Last 2Y Investments'], # Text
    'inv_amount_average': ['Average Invested Amount',[
        ['<25K€', '25k€-50k€', '50k€-100k€', '100k€-500k€', '>500k€', ],
        ['<0.5K€', '0.5k€-1k€', '1k€-5k€', '5k€-10k€', '10k€-50k€', '>50k€'],
    ],], # Not same labels
    'inv_capital_stake': ['Typical Stake', ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%', '>51%'],], # Only for BA
    'inv_dd_duration': ['Analysis Duration',  [
        ['Less than 1 month', '1-6 months', '7-12 months', '13-18 months', 'More than 18 months'],
        ['Less than 1 hour', '1 to 2 hours', '3 to 5 hours', '6 to 10 hours', 'More than 10 hours'],
    ],], # Not same labels
    'inv_criteria': ['Investing Criteria',  [
        # ['Alignment with my experience and expertise', 'Business plan', 'Scalability/Market potential', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
        # ['Business plan', 'Crowd opinion', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
        ['Expertise', 'Business plan', 'Scalability', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
        ['Business plan', 'Crowd opinion', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    ],], # Not same labels multiple
    'inv_sectors': ['Sectors', 
        # ['Information technology (software, internet, IT services, etc.)', 'Finance', 'Healthcare / Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
        ['IT', 'Finance', 'Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    ],
    'inv_process': ['Investing Process',
        # ['I invest alone (as an individual investor)', 'I invest as part of an angel syndicate or group', 'I invest through venture capital funds', 'I invest through crowdfunding platforms', 'Other'],
        ['Alone', 'Syndicate/group', 'Venture capital', 'Crowdfunding platforms', 'Other'],
    ], # Only BA
    'inv_visits': ['Number of Visits', ['0 per year', '1-2 per year', '3-6 per year', '7-12 per year', 'More than 12 per year'],], # Only BA
    'non_fin_contr': ['Non-Financial Contribution',
        # ['Strategic guidance', 'Networking opportunities', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring and accountability', 'Other',],
        ['Strategy', 'Networking', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring', 'Other',],
    ], # Only BA
    'child_bias': ['Children Effect', ['1 - Not at all', '2', '3', '4', '5 - Very much',],], # Only BA
    'inv_goal': ['Investment Goal',
        # ['Capital preservation (low risk, low return)', 'Income generation (moderate risk, moderate return)', 'Growth (high risk, high return)', 'Speculative (very high risk, very high return)'],
        ['Capital preservation', 'Income generation', 'Growth', 'Speculative'],
    ], 
    'inv_exit': ['Exit', [
        # ['No, I have not sold my investments yet', 'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
        # ['No, I have not sold my investments yet', "Yes, through the platform’s secondary market",  'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
        ['Not sold yet', 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
        ['Not sold yet', "Platform’s secondary market", 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
    ] ], # Not same labels
    'inv_return': ['Investment Returns',
        # ['I have no idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
        ['No idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    ],
    'women_investing': ['Women Investing Knowledge', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    'women_involvement': ['Women Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    'women_influence': [ 'Women Influence', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    'women_diff_eval_process': [ 'Women Evaluation Process', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    'women_risk_averse': ['Women Risk Aversion', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    'women_evolution_involvement': ['Women Evolution Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    'women_evolution_neg_stereotype': ['Women Stereotype Evolution', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    'mother_needs_consideration': ["Consideration of Mothers' Needs", ['1-Not at all', '2', '3', '4', '5-Completely',]],
    'women_reco': ['Recommendations for Attracting Women',[
        # ['Increase visibility of successful women business angels', 'Offer targeted networking events for women', 'Highlight women-led projects', 'Offer training or mentorship programs for aspiring women investors', 'Establish women-focused investment funds or syndicates', 'Other'],
        # ['Create events or online communities where women can network', 'Ensure the platform is user-friendly and inclusive', 'Highlight women-led projects', 'Offer training programs', 'Showcase successful women crowd-investors in the investment space', 'Other']
        ['Communication on success', 'Networking', 'Communication on projects', 'Training', 'Specific funds', 'Other'],
        ['Networking', 'User-Friendliness','Communication on projects', 'Training', 'Communication on success', 'Other'],
    ]],
    'women_led_inv': ['Number of Women-Led Investments', [
        ['None', '1-2', '3-5', '6-10', '11-20', '>20'],
        ['None', '1-2', '3-5', '6-10', '>10'],
    ]],
}

ORDINAL_COLS=[
    'inv_time',
    'inv_num',
    'inv_amount_average',
    'inv_capital_stake',
    'inv_dd_duration',
    'inv_visits',
    'child_bias',
    'inv_goal',
    'inv_return',
    'women_investing',
    'women_involvement',
    'women_influence',
    'women_diff_eval_process',
    'women_risk_averse',
    'women_evolution_involvement',
    'women_evolution_neg_stereotype',
    'mother_needs_consideration',
    'women_led_inv',
]

MUL_COLS=[
    'inv_motivation',
    'ba_group',
    'ba_group_name',
    'ba_group_reason',
    'inv_criteria',
    'inv_sectors',
    'inv_process',
    'non_fin_contr',
    'inv_exit',
    'inv_platform',
    'women_reco',
]

DIFF_COLS=[
    'inv_motivation',
    'inv_amount_average',
    'inv_dd_duration',
    'inv_criteria',
    'inv_exit',
    'inv_platform',
]

BA_ONLY_COLS=[
    'ba_group',
    'ba_group_name',
    'ba_group_reason',
    'inv_capital_stake',
    'inv_process',
    'inv_visits',
    'non_fin_contr',
    'child_bias',
]

CI_ONLY_COLS=[
    'inv_platform',
]

GENDER_COLS=[
    'women_investing',
    'women_involvement',
    'women_influence',
    'women_diff_eval_process',
    'women_risk_averse',
    'women_evolution_involvement',
    'women_evolution_neg_stereotype',
    'mother_needs_consideration',
    'women_reco',
    'women_led_inv',
]

Functions

In [6]:
############### Functions ###############
import pandas as pd 
import scipy
from scipy.stats import mannwhitneyu
import pandas as pd 

# Returns the statistics in funcs on groups and variables columns
def des_stats(df, funcs, group_label, columns):
    df_results=pd.DataFrame()
    for column in columns:
        df_results=pd.concat([df_results,df.groupby(group_label)[column].agg(funcs).reset_index().assign(var=column)])
    return df_results

# def for_ba

# Mode or most frequent result 
def mode(x):
    return scipy.stats.mode(x.dropna())[0]

# Total number of records
def total(x):
    return len(x)

# Count the number of NA
def count_na(x):
    return x.isna().sum()

# Percentage of NA
def percent_na(x):
    return x.isna().sum()/len(x)*100

# Frequency table

def freq_table(df, column, start_index, category_labels):
    df_results=pd.DataFrame(columns=['Response', 'Frequency', 'Percentage'])
    for index, category_label in enumerate(category_labels):
        df_temp= pd.DataFrame([[category_label, len(df[df[column]==(index+start_index)]), round(100*len(df[df[column]==(index+start_index)])/len(df[column]),2)]], columns=df_results.columns)
        df_results=pd.concat([df_results,df_temp])
    df_results=pd.concat([df_results, pd.DataFrame([['NA', df[column].isna().sum(), round(100*df[column].isna().sum()/len(df[column]),2)]], columns=df_results.columns)])
    df_results=pd.concat([df_results, pd.DataFrame([['Total', len(df[column]), 100.00]], columns=df_results.columns)])
    return df_results

def mw_test(df,columns, condition_label, condition_values):
    result_columns=['var', 'u_stat', 'pvalue']
    # df_results=pd.DataFrame(columns=result_columns)
    df_results=pd.DataFrame()
    for column in columns:
        
        df_temp=df.dropna(subset=[column, condition_label]) # Keep only non nan

        groups=[]
        for i in range(len(condition_values)):
            groups.append(df_temp[df_temp[condition_label]==condition_values[i]].loc[:,column].array)

        u, p=mannwhitneyu(groups[0], groups[1], alternative='two-sided')
        df_results=pd.concat([df_results, pd.DataFrame([[column, u, p]], columns=result_columns)])
    
    return df_results

def des_stats_table(df, funcs, group_label, columns):
    df_temp=des_stats(df, funcs, group_label, columns)
    df_temp=df_temp.reindex(columns=['var']+[col for col in df_temp.columns if col != 'var']) # Position 'var' column in first place
    values=df[group_label].unique()
    df_results=df_temp[df_temp[group_label]==values[0]].merge(df_temp[df_temp[group_label]==values[1]], left_on='var', right_on='var', suffixes=('_'+str(values[0]),'_'+str(values[1])))
    # Merge with WM test
    df_results=df_results.merge(mw_test(df,columns,group_label,values).loc[:,['var', 'pvalue']], left_on='var', right_on='var',)
    
    return df_results

# Descriptive Statistics and Whitney Mann U Test

In [ ]:
# Descriptive statistics
# Gender
import pandas as pd

# Basic statistics
df=df_load
# df=df_load[df_load['survey']=='BA'] # BA only
# df=df_load[df_load['survey']=='CI'] # CI only

funcs=[mode, 'median', 'mean', 'std', 'count',]
group_label='gender_male'

# Ordinal, not multiple options, not in gender
# variable labels can be different for BAs and CIs
columns=[]
for col in COLS:
    if (
        (col in ORDINAL_COLS) 
        & (not col in MUL_COLS) 
        & (not col in GENDER_COLS)
    ):
        columns.append(col)

df_results=des_stats_table(df, funcs, group_label, columns)
df_results

,var,gender_male_2.0,mode_2.0,median_2.0,mean_2.0,std_2.0,count_2.0,gender_male_1.0,mode_1.0,median_1.0,mean_1.0,std_1.0,count_1.0,pvalue
0,inv_time,2.0,5.0,3.0,3.372093,1.337658,86,1.0,3.0,3.0,3.064516,1.093480,31,0.261672
1,inv_num,2.0,3.0,3.0,3.423529,1.483051,85,1.0,3.0,3.0,3.375000,1.539795,32,0.965182
2,inv_amount_average,2.0,1.0,2.0,2.500000,1.295408,74,1.0,1.0,1.0,1.814815,1.177907,27,0.011706
3,inv_capital_stake,2.0,1.0,2.0,1.841270,0.987116,63,1.0,1.0,1.0,1.521739,0.897956,23,0.105298
4,inv_dd_duration,2.0,3.0,3.0,2.683544,0.725794,79,1.0,3.0,3.0,2.612903,0.667204,31,0.677881
5,inv_visits,2.0,2.0,2.0,2.569231,1.015031,65,1.0,2.0,2.0,2.260870,0.963771,23,0.219112
6,child_bias,2.0,1.0,1.0,1.893617,1.146527,47,1.0,1.0,1.0,2.166667,1.465285,18,0.561968
7,inv_goal,2.0,3.0,3.0,2.775000,0.655551,80,1.0,3.0,3.0,2.483871,0.676805,31,0.056241
8,inv_return,2.0,5.0,4.0,4.000000,1.433994,72,1.0,5.0,4.0,3.777778,1.476309,27,0.530877


In [ ]:
# Descriptive statistics
# Investor type
import pandas as pd

# Basic statistics
df=df_load
# df=df_load[df_load['gender_male']==1] # Women only
# df=df_load[df_load['gender_male']==2] # Men only

funcs=[mode, 'median', 'mean', 'std', 'count',]
group_label='survey'

# Ordinal, not multiple options, not in gender
# variable labels can be different for BAs and CIs
# May have N/A because BA only or CI only
columns=[]
for col in COLS:
    if (
        (col in ORDINAL_COLS) 
        & (not col in MUL_COLS) 
        & (not col in GENDER_COLS)
    ):
        columns.append(col)

df_results=des_stats_table(df, funcs, group_label, columns)
df_results

C:\Users\Dell\AppData\Local\Temp\ipykernel_7040\3030130470.py:18: SmallSampleWarning:

One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.

C:\Users\Dell\AppData\Local\Temp\ipykernel_7040\3030130470.py:18: SmallSampleWarning:

One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.

C:\Users\Dell\AppData\Local\Temp\ipykernel_7040\3030130470.py:18: SmallSampleWarning:

One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.

C:\Users\Dell\AppData\Local\Temp\ipykernel_7040\3030130470.py:55: SmallSampleWarning:

One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.



,var,survey_BA,mode_BA,median_BA,mean_BA,std_BA,count_BA,survey_CI,mode_CI,median_CI,mean_CI,std_CI,count_CI,pvalue
0,inv_time,BA,3.0,4.0,3.563830,1.187375,94,CI,2.0,2.0,2.173913,1.029217,23,5.037791e-06
1,inv_num,BA,3.0,4.0,3.760870,1.329046,92,CI,1.0,2.0,2.153846,1.347362,26,6.397449e-07
2,inv_amount_average,BA,1.0,2.0,2.306818,1.271780,88,CI,2.0,2.0,2.384615,1.502135,13,9.495629e-01
3,inv_capital_stake,BA,1.0,1.0,1.755814,0.969367,86,CI,NaN,NaN,NaN,NaN,0,NaN
4,inv_dd_duration,BA,3.0,3.0,2.688889,0.611582,90,CI,3.0,3.0,2.550000,1.050063,20,6.004409e-01
5,inv_visits,BA,2.0,2.0,2.488636,1.005666,88,CI,NaN,NaN,NaN,NaN,0,NaN
6,child_bias,BA,1.0,1.0,1.969231,1.237048,65,CI,NaN,NaN,NaN,NaN,0,NaN
7,inv_goal,BA,3.0,3.0,2.766667,0.671239,90,CI,2.0,2.0,2.380952,0.589592,21,1.749176e-02
8,inv_return,BA,5.0,4.0,4.011628,1.409999,86,CI,5.0,3.0,3.461538,1.613246,13,3.009928e-01


Radar Charts

In [ ]:
# Radar chart
import plotly.graph_objects as go

def radar(df, group_label, columns, stat_func, traces, value_range):

    FUNCS=[mode, 'median', 'mean', 'std', 'count',]
    
    # Build column_names and categories
    column_labels=[]
    categories=[]
    for i in range(len(columns)):
         column_labels.append(columns[i])
         categories.append(COLS[columns[i]][0])
         
    # Get statistics
    df_stats=des_stats(df,FUNCS, group_label, column_labels)

    fig = go.Figure()

    # Traces
    for i in range(2):
        fig.add_trace(go.Scatterpolar(
                r=df_stats[df_stats[group_label]==traces[i][0]][stat_func],
                theta=categories,
                fill='toself',
                opacity=0.5,
                line_shape='spline',
                name=traces[i][1]
        ))
        
    # Formatting

    axis_color = "rgba(108, 122, 137,1)"
    fig.update_layout(
    polar=dict(
        bgcolor="rgba(0, 0, 0, 0)",
        angularaxis = dict(
        linewidth = 2,
        showline=True,
        linecolor =axis_color,
        gridcolor =axis_color,
    ),
        
    radialaxis = dict(
        side = "counterclockwise",
        showline = False,
        linewidth = 1,
        gridcolor =axis_color,
        gridwidth = 1,
        dtick = 1,
        range=value_range,
    )
    ),
    
    legend=dict(
        yanchor="bottom",
        y=-0.4,
        xanchor="center",
        x=0.5,
        bordercolor=axis_color,
        borderwidth=1,
    ),
    )

    fig.show()
    

In [ ]:
# Variable means by gender - Business angel & crowdfunding investors

df=df_load.copy()

group_label='gender_male'

# Ordinal, not multiple options, not in gender, not in BA only, not in CI only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      & (not col in BA_ONLY_COLS)
      & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

stat_func='mean' 

traces=[
  [1, 'Women'],
  [2, 'Men'],
]

value_range=[0,5]

print('Variable means by gender - Business angel & crowdfunding investors')
radar(df, group_label, columns, stat_func, traces, value_range)

Variable means by gender - Business angel & crowdfunding investors


In [26]:
# Variable means by gender - Business angel only

df=df_load[df_load['survey']=='BA']

group_label='gender_male'

# Ordinal, not multiple options, not in gender, not in CI only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      # & (not col in BA_ONLY_COLS)
      & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

traces=[
  [1, 'Women'],
  [2, 'Men'],
]

value_range=[0,5]

radar(df, group_label, columns, stat_func, traces, value_range)

In [27]:
# Variable means by gender - Crowdfunding investor only

df=df_load[df_load['survey']=='CI']

group_label='gender_male'

# Ordinal, not multiple options, not in gender, not in BA only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      & (not col in BA_ONLY_COLS)
    #   & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

stat_func='mean' 

traces=[
  [1, 'Women'],
  [2, 'Men'],
]

value_range=[0,5]

radar(df, group_label, columns, stat_func, traces, value_range)

In [28]:
# Variable means by investor type - Women and men

df=df_load.copy()

group_label='survey'

# Ordinal, not multiple options, not in gender, not in BA only, not in CI only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      & (not col in BA_ONLY_COLS)
      & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

stat_func='mean' 

traces=[
  ['BA', 'Business Angels'],
  ['CI', 'Crowdfunding Investors'],
]

value_range=[0,5]

radar(df, group_label, columns, stat_func, traces, value_range)

In [29]:
# Variable means by investor type - Women only

df=df_load[df_load['gender_male']==1]

group_label='survey'

# Ordinal, not multiple options, not in gender, not in BA only, not in CI only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      & (not col in BA_ONLY_COLS)
      & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

stat_func='mean' 

traces=[
  ['BA', 'Business Angels'],
  ['CI', 'Crowdfunding Investors'],
]

value_range=[0,5]

radar(df, group_label, columns, stat_func, traces, value_range)

In [51]:
# Variable means by investor type - Men only

df=df_load[df_load['gender_male']==2]

group_label='survey'

# Ordinal, not multiple options, not in gender, not in BA only, not in CI only, add 'women_led_inv'
columns=[]
for col in COLS:
    if (
      (col in ORDINAL_COLS)
      & (not col in MUL_COLS)
      & (not col in GENDER_COLS)
      & (not col in BA_ONLY_COLS)
      & (not col in CI_ONLY_COLS)
& (not col in DIFF_COLS)
    ):
        columns.append(col)

columns.append('women_led_inv')

stat_func='mean' 

traces=[
  ['BA', 'Business Angels'],
  ['CI', 'Crowdfunding Investors'],
]

value_range=[0,5]

radar(df, group_label, columns, stat_func, traces, value_range)

Frequency Tables

In [148]:
map_table=[
        [1,'Within the last year'],
        [2,'2-3 years ago'],
        [3,'4-6 years ago'],
        [4,'7-9 years ago'],
        [5,'10 or more years ago '],
]

title='Investment Experience'
df=df_load
df_display=freq_table(df, 'inv_time', 1, map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Women'
df=df_load[df_load['gender_male']==1]
df_display=freq_table(df, 'inv_time', 1, map_table)
df_display.style \
        .set_caption(title)
        
title='Investment Experience - Men'
df=df_load[df_load['gender_male']==2]
df_display=freq_table(df, 'inv_time', 1, map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Business Angels'
df=df_load[df_load['survey']=='BA']
df_display=freq_table(df, 'inv_time', 1, map_table)
df_display.style \
        .set_caption(title)

title='Investment Experience - Crowdfunding Investors'
df=df_load[df_load['survey']=='CI']
df_display=freq_table(df, 'inv_time', 1, map_table)
df_display.style \
        .set_caption(title)
        

C:\Users\Dell\AppData\Local\Temp\ipykernel_15800\3030130470.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Dell\AppData\Local\Temp\ipykernel_15800\3030130470.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

C:\Users\Dell\AppData\Local\Temp\ipykernel_15800\3030130470.py:38: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes

,Response,Frequency,Percentage
0,"[1, 'Within the last year']",7,10.770000
0,"[2, '2-3 years ago']",8,12.310000
0,"[3, '4-6 years ago']",5,7.690000
0,"[4, '7-9 years ago']",3,4.620000
0,"[5, '10 or more years ago ']",0,0.000000
0,NA,42,64.620000
0,Total,65,100.000000


In [50]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def hist(df, cols, start_range):
    for col in cols:
        main_title=cols[col][0]
        x_title=''
        x_label=col
        y_title='Frequency'
        category_orders=cols[col][1]
        category_range=range(start_range, len(category_orders)+1)
        
        # print(col[0])

        ######################## Histograms - One group ########################

        groups={
            'gender_male':{
                'title': main_title + ' - Gender',
                'names': ['Women', 'Men'],
                'values': [1, 2],
            },
            'survey':{
                'title': main_title + ' - Investor Type',
                'names': ['Business Angel', 'Crowdfunding Investor'],
                'values': ['BA', 'CI'],
            },
        }
        for group in groups:
            fig = go.Figure()
            for i in range(len(groups[group]['values'])):  
                print('gender: {}'.format(groups['gender_male']['values'][i]))
                print('x')    
                x=df[df[group]==groups[group]['values'][i]][x_label].replace(category_range, category_orders)
                print(x)
                fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups[group]['names'][i]))
        
            # Layout
            print(groups[group]['title'])
            fig.update_layout(
                # title=groups[group]['title'],
                xaxis={
                    'title': x_title,
                    'showline': True,
                    'linewidth': 1,
                    'linecolor': 'black',
                    # 'mirror': True,
                    'categoryorder': 'array',
                    'categoryarray': category_orders,
                },
                yaxis={
                    'title': y_title,
                    'showline': True,
                    'linewidth': 1,
                    'linecolor': 'black',
                    # 'mirror': True,
                },
                legend={
                'yanchor': 'bottom',
                'y': -0.25,
                'xanchor': 'center',
                'x': 0.5,
                'orientation': 'h',
                },
            )

            fig.show()
        
        ######################## Histograms - Cross Groups ########################

        groups={
            'gender_male':{
                'title': main_title + ' - Gender',
                'names': ['Women', 'Men'],
                'values': [1, 2],
                'marker_color': ['red', 'blue']
            },
            'survey':{
                'title': main_title + ' - Investor Type',
                'names': ['Business Angel', 'Crowdfunding Investor'],
                'values': ['BA', 'CI'],
                'opacity': [0.75, 0.5] 
            },
        }
        title = main_title + ' - Gender & Investor Type'
        fig = go.Figure()
        
        for i in range(len(groups['gender_male']['values'])):    
            for j in range(len(groups['survey']['values'])):  
                x=df[(df['gender_male']==groups['gender_male']['values'][i]) & (df['survey']==groups['survey']['values'][j])][x_label].replace(category_range, category_orders)
                fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups['gender_male']['names'][i]+' & '+groups['survey']['names'][j], marker_color=groups['gender_male']['marker_color'][i], opacity=groups['survey']['opacity'][j]))

        # Layout
        print(title)
        fig.update_layout(
            # title=title,
            xaxis={
                'title': x_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
                'categoryorder': 'array',
                'categoryarray': category_orders,
            },
            yaxis={
                'title': y_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
            },
            legend={
                'yanchor': 'bottom',
                'y': -0.25,
                'xanchor': 'center',
                'x': 0.5,
                'orientation': 'h',
            },
        )

        fig.show()
    

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def hist(traces, category_orders, x_title, y_title, y_tickformat, histnorm):
    fig = go.Figure()
    for trace in traces:
        fig.add_trace(go.Histogram(histnorm=histnorm, x=trace['x'], name=trace['name'], marker_color=trace['marker_color'], opacity=trace['opacity']))
    
    # Layout
    fig.update_layout(
        # title=groups[group]['title'],
        xaxis={
            'title': x_title,
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
            # 'mirror': True,
            'categoryorder': 'array',
            'categoryarray': category_orders,
        },
        yaxis={
            'title': y_title,
            'showline': True,
            'linewidth': 1,
            'linecolor': 'black',
            # 'mirror': True,
            'tickformat': y_tickformat,
        },
        legend={
        'yanchor': 'bottom',
        'y': -0.25,
        'xanchor': 'center',
        'x': 0.5,
        'orientation': 'h',
        },
    )

    fig.show()

def dist_ana(df, cols, start_range, groups, y_title, y_tickformat, histnorm):
    for col in cols:
        
        # Consolidated Basis
        for group in groups:
            category_range=range(start_range, start_range+len(cols[col][1]))
            category_orders=cols[col][1]
            
            if (group=='gender_male'):
                title=cols[col][0]+ ' - Gender - BA & CI'
            elif (group=='survey'):
                title=cols[col][0]+ ' - Investor Type - Women & Men'
                
            traces=[]
            for i in range(len(groups)):
                df_temp=df[df[group]==groups[group][i][0]]
                traces.append({
                    'x': df_temp[col].replace(category_range, category_orders),
                    'name': groups[group][i][1],
                    'marker_color': groups[group][i][2],
                    'opacity': groups[group][i][3],
                })
            x_title=''
        
        
            
            # Display
            print(title)
            hist(traces, category_orders, x_title, y_title, y_tickformat, histnorm)
            
            # Gender BA only   
            # if (not col in CI_ONLY_COLS) & (not col in DIFF_COLS): 
            #     #Inputs
            #     if (group=='gender_male'):
            #         title=cols[col][0]+ ' - Gender - BA & CI'
            #     elif (group=='survey'):
            #         title=cols[col][0]+ ' - Investor Type - Women & Men'
            #     traces=[]
            #     for i in range(len(groups)):
            #         df_temp=df[
            #             (df[group]==groups[group][i][0])
            #             ]
            #         traces.append({
            #             'x': df_temp[col].replace(category_range, category_orders),
            #             'name': groups[group][i][1],
            #             'marker_color': groups[group][i][2],
            #             'opacity': groups[group][i][3],
            #         })
            #     x_title=''
                
            #     # Display
            #     print(title)
            #     hist(traces, category_orders, x_title, y_title, y_tickformat, histnorm)
            
            
            

# Execution
df=df_load.copy()
cols={
    'inv_time': ['Investor Experience', ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],],
}
groups={
    'gender_male':[
        [1, 'Women', 'red', 0.5],
        [2, 'Men', 'blue', 0.5],
    ],
    'survey': [
        ['BA', 'Business Angel', 'green', 0.5],
        ['CI', 'Crowdfunding Investor', 'purple', 0.5],
    ],
}
start_range=1
histnorm='percent'
y_title='Percentage'
y_tickformat=''

dist_ana(df, cols, start_range, groups, y_title, y_tickformat, histnorm)


Investor Experience - Gender - BA & CI


Investor Experience - Investor Type - Women & Men


In [ ]:
def hist_temp(df, cols, start_range):
    for col in cols:
        main_title=cols[col][0]
        x_title=''
        x_label=col
        y_title='Frequency'
        category_orders=cols[col][1]
        category_range=range(start_range, len(category_orders)+1)
        
        ######################## Histograms - Gender ########################

        # groups={
        #     'gender_male':{
        #         'title': main_title + ' - Gender',
        #         'names': ['Women', 'Men'],
        #         'values': [1, 2],
        #     },
        #     'survey':{
        #         'title': main_title + ' - Investor Type',
        #         'names': ['Business Angel', 'Crowdfunding Investor'],
        #         'values': ['BA', 'CI'],
        #     },
        # }
        
        genders=[
            [1, 2],
            ['Women', 'Men'],
        ]
        
        investor_types=[
            ['BA', 'CI'],
            ['Business Angel', 'Crowdfunding Investor'],
        ]
        
        for group in groups:
            fig = go.Figure()
            for i in range(len(groups[group]['values'])):  
                print('gender: {}'.format(groups['gender_male']['values'][i]))
                print('x')    
                x=df[df[group]==groups[group]['values'][i]][x_label].replace(category_range, category_orders)
                print(x)
                fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups[group]['names'][i]))
        
        
        
            # Layout
            print(groups[group]['title'])
            fig.update_layout(
                # title=groups[group]['title'],
                xaxis={
                    'title': x_title,
                    'showline': True,
                    'linewidth': 1,
                    'linecolor': 'black',
                    # 'mirror': True,
                    'categoryorder': 'array',
                    'categoryarray': category_orders,
                },
                yaxis={
                    'title': y_title,
                    'showline': True,
                    'linewidth': 1,
                    'linecolor': 'black',
                    # 'mirror': True,
                },
                legend={
                'yanchor': 'bottom',
                'y': -0.25,
                'xanchor': 'center',
                'x': 0.5,
                'orientation': 'h',
                },
            )

            fig.show()
        
        ######################## Histograms - Cross Groups ########################

        groups={
            'gender_male':{
                'title': main_title + ' - Gender',
                'names': ['Women', 'Men'],
                'values': [1, 2],
                'marker_color': ['red', 'blue']
            },
            'survey':{
                'title': main_title + ' - Investor Type',
                'names': ['Business Angel', 'Crowdfunding Investor'],
                'values': ['BA', 'CI'],
                'opacity': [0.75, 0.5] 
            },
        }
        title = main_title + ' - Gender & Investor Type'
        fig = go.Figure()
        
        for i in range(len(groups['gender_male']['values'])):    
            for j in range(len(groups['survey']['values'])):  
                x=df[(df['gender_male']==groups['gender_male']['values'][i]) & (df['survey']==groups['survey']['values'][j])][x_label].replace(category_range, category_orders)
                fig.add_trace(go.Histogram(histnorm='percent', x=x, name=groups['gender_male']['names'][i]+' & '+groups['survey']['names'][j], marker_color=groups['gender_male']['marker_color'][i], opacity=groups['survey']['opacity'][j]))

        # Layout
        print(title)
        fig.update_layout(
            # title=title,
            xaxis={
                'title': x_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
                'categoryorder': 'array',
                'categoryarray': category_orders,
            },
            yaxis={
                'title': y_title,
                'showline': True,
                'linewidth': 1,
                'linecolor': 'black',
                # 'mirror': True,
            },
            legend={
                'yanchor': 'bottom',
                'y': -0.25,
                'xanchor': 'center',
                'x': 0.5,
                'orientation': 'h',
            },
        )

        fig.show()

Distribution analysis

In [ ]:
# Distribution for unique choice and same list of options

df=df_load.copy()

cols={
    'inv_time': ['Investor Experience', ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],],
}


start_index=1

hist(df, cols, start_index)


x
2       7-9 years
3       7-9 years
10      4-6 years
12        <1 year
14      2-3 years
16      4-6 years
22      4-6 years
24      7-9 years
27      4-6 years
43      7-9 years
45      4-6 years
49            NaN
52      4-6 years
56      4-6 years
57      7-9 years
61      4-6 years
64            NaN
67            NaN
68      4-6 years
74      2-3 years
78            NaN
80      7-9 years
84      4-6 years
85      7-9 years
98            NaN
112     7-9 years
114     7-9 years
117    > 10 years
120    > 10 years
124           NaN
127     4-6 years
132       <1 year
135           NaN
136       <1 year
137           NaN
142           NaN
144     4-6 years
151           NaN
154           NaN
155     4-6 years
161           NaN
164           NaN
171           NaN
180       <1 year
181     2-3 years
184           NaN
187           NaN
Name: inv_time, dtype: object
x
0         <1 year
1         <1 year
4       2-3 years
5      > 10 years
6       4-6 years
          ...    
178         

x
0         <1 year
1         <1 year
2       7-9 years
3       7-9 years
4       2-3 years
          ...    
118     7-9 years
119    > 10 years
120    > 10 years
121    > 10 years
122     2-3 years
Name: inv_time, Length: 123, dtype: object
x
123    2-3 years
124          NaN
125    2-3 years
126    7-9 years
127    4-6 years
         ...    
183          NaN
184          NaN
185          NaN
186          NaN
187          NaN
Name: inv_time, Length: 65, dtype: object
Investor Experience - Investor Type


gender: 1
gender: 1
gender: 2
gender: 2
Investor Experience - Gender & Investor Type


In [ ]:
# Distribution for unique choice and same list of options

df=df_load.copy()

columns=[
    ['inv_time', 'Investor Experience', ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],],  
    # ['inv_motivations', 'Motivations', [
    #     # ['Support innovative startups or entrepreneurs', 'Potential high returns on investments', 'To diversify my investment portfolio', 'Recommendation from friends', 'Benefits from tax breaks', 'My financial capacity', 'My family background', 'Other' ],
    #     # ['Desire to support innovative startups or entrepreneurs', 'Potential high returns on investments', 'Diversifying my investment portfolio', 'Recommendation from friends or media influence', 'Benefits from tax breaks', 'Business reasons (you could be a supplier, etc.)', 'Other' ],
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Financial capacity', 'Family background', 'Other' ],
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Business reasons', 'Other' ],
    # ],], # Not same labels
    # ['ba_group', 'Part of BA Network', ['Yes', 'No'],], # Boolean Only BA
    # ['ba_group_name', 'BA Network'], # Only BA
    # ['ba_group_reason', 'Part of BA Network Motivations',
    #   # ['To gain access to a larger number of investment opportunities', 'To invest in mutual funds set up by the network', 'To leverage the experience of other network members', 'To benefit from educational or training sessions', 'To expand my personal and professional network', 'To coinvest with other investors', 'Other',],
    #   ['More opportunities', 'Network funds', "Members' experience", 'Training', 'Expand network', 'Coinvest', 'Other',],
    # ], # Only BA  
    ['inv_num', 'Number of Investments', ['None', '1-2', '3-5', '6-10', '11-20', '>20'],],
    # ['inv_companies_last2y', 'Last 2Y Investments'], # Text
    # ['inv_amount_average', 'Average Invested Amount',
    #    [
    #     ['<25K€', '25k€-50k€', '50k€-100k€', '100k€-500k€', '>500k€', ],
    #     ['<0.5K€', '0.5k€-1k€', '1k€-5k€', '5k€-10k€', '10k€-50k€', '>50k€'],
    # ],], # Not same labels
    # ['inv_capital_stake', 'Typical Stake', ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%', '>51%'],], # Only for BA
    # ['inv_dd_duration', 'Analysis Duration',  [
    #     ['Less than 1 month', '1-6 months', '7-12 months', '13-18 months', 'More than 18 months'],
    #     ['Less than 1 hour', '1 to 2 hours', '3 to 5 hours', '6 to 10 hours', 'More than 10 hours'],
    # ],], # Not same labels
    # ['inv_criteria', 'Investing Criteria',  [
    #     # ['Alignment with my experience and expertise', 'Business plan', 'Scalability/Market potential', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    #     # ['Business plan', 'Crowd opinion', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    #     ['Expertise', 'Business plan', 'Scalability', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    #     ['Business plan', 'Crowd opinion', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    # ],], # Not same labels multiple
    # ['inv_sectors', 'Sectors', 
    #     # ['Information technology (software, internet, IT services, etc.)', 'Finance', 'Healthcare / Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    #     ['IT', 'Finance', 'Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    # ],
    # ['inv_process', 'Investing Process',
    #     # ['I invest alone (as an individual investor)', 'I invest as part of an angel syndicate or group', 'I invest through venture capital funds', 'I invest through crowdfunding platforms', 'Other'],
    #     ['Alone', 'Syndicate/group', 'Venture capital', 'Crowdfunding platforms', 'Other'],
    # ], # Only BA
    # ['inv_visits', 'Number of Visits', ['0 per year', '1-2 per year', '3-6 per year', '7-12 per year', 'More than 12 per year'],], # Only BA
    # ['non_fin_contr', 'Non-Financial Contribution',
    #     # ['Strategic guidance', 'Networking opportunities', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring and accountability', 'Other',],
    #     ['Strategy', 'Networking', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring', 'Other',],
    # ], # Only BA
    # ['child_bias', 'Children Effect', ['1 - Not at all', '2', '3', '4', '5 - Very much',],], # Only BA
    # ['inv_goal', 'Investment Goal',
    #     # ['Capital preservation (low risk, low return)', 'Income generation (moderate risk, moderate return)', 'Growth (high risk, high return)', 'Speculative (very high risk, very high return)'],
    #     ['Capital preservation', 'Income generation', 'Growth', 'Speculative'],
    # ], 
    # ['inv_exit', 'Exit', [
    #     # ['No, I have not sold my investments yet', 'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
    #     # ['No, I have not sold my investments yet', "Yes, through the platform’s secondary market",  'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
    #     ['Not sold yet', 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
    #     ['Not sold yet', "Platform’s secondary market", 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
    # ] ], # Not same labels
    # ['inv_return', 'Investment Returns',
    #     # ['I have no idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    #     ['No idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    # ],
    # ['women_investing', 'Women Investing Knowledge', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    # ['women_involvement', 'Women Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    # ['women_influence', 'Women Influence', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_diff_eval_process', 'Women Evaluation Process', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_risk_averse', 'Women Risk Aversion', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_evolution_involvement', 'Women Evolution Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_evolution_neg_stereotype', 'Women Stereotype Evolution', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['mother_needs_consideration', "Consideration of Mothers' Needs", ['1-Not at all', '2', '3', '4', '5-Completely',]],
    # ['women_reco', 'Recommendations for Attracting Women',[
    #     # ['Increase visibility of successful women business angels', 'Offer targeted networking events for women', 'Highlight women-led projects', 'Offer training or mentorship programs for aspiring women investors', 'Establish women-focused investment funds or syndicates', 'Other'],
    #     # ['Create events or online communities where women can network', 'Ensure the platform is user-friendly and inclusive', 'Highlight women-led projects', 'Offer training programs', 'Showcase successful women crowd-investors in the investment space', 'Other']
    #     ['Communication on success', 'Networking', 'Communication on projects', 'Training', 'Specific funds', 'Other'],
    #     ['Networking', 'User-Friendliness','Communication on projects', 'Training', 'Communication on success', 'Other'],
    # ]],
    # ['women_led_inv', 'Number of Women-Led Investments', [
    #     ['None', '1-2', '3-5', '6-10', '11-20', '>20'],
    #     ['None', '1-2', '3-5', '6-10', '>10'],
    # ],
]

start_index=1

hist(df, columns, start_index)


Investor Experience - Gender


Investor Experience - Investor Type


Investor Experience - Gender & Investor Type


Number of Investments - Gender


Number of Investments - Investor Type


Number of Investments - Gender & Investor Type


Number of Women-Led Investments - Gender


Number of Women-Led Investments - Investor Type


Number of Women-Led Investments - Gender & Investor Type


In [54]:
# Distribution for unique choice and same list of options

df=df_load.copy()

# columns=[
#     'inv_time',
#     # 'inv_motivation', # Multiple
#     'ba_group', # Boolean
#     # 'ba_group_name', # Not same labels
#     # 'ba_group_reason', # Multiple
#     'inv_num',
#     # 'inv_companies_last2y', # Text
#     # 'inv_amount_average', # Not same labels
#     'inv_capital_stake',
#     # 'inv_dd_duration', # Not same labels
#     # 'inv_criteria', # Multiple Not same labels
#     # 'inv_sectors', # Multiple
#     # 'inv_process',
#     'inv_visits',
#     # 'non_fin_contr', # Not the same labels
#     'child_bias', 
#     'inv_goal',
#     # 'inv_exit', # Multiple, Not the same labels
#     'inv_return',

# ]

columns=[
    ['inv_time', 'Investor Experience', ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],],  
    # ['inv_motivations', 'Motivations', [
    #     # ['Support innovative startups or entrepreneurs', 'Potential high returns on investments', 'To diversify my investment portfolio', 'Recommendation from friends', 'Benefits from tax breaks', 'My financial capacity', 'My family background', 'Other' ],
    #     # ['Desire to support innovative startups or entrepreneurs', 'Potential high returns on investments', 'Diversifying my investment portfolio', 'Recommendation from friends or media influence', 'Benefits from tax breaks', 'Business reasons (you could be a supplier, etc.)', 'Other' ],
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Financial capacity', 'Family background', 'Other' ],
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Business reasons', 'Other' ],
    # ],], # Not same labels
    # ['ba_group', 'Part of BA Network', ['Yes', 'No'],], # Boolean Only BA
    # ['ba_group_name', 'BA Network'], # Only BA
    # ['ba_group_reason', 'Part of BA Network Motivations',
    #   # ['To gain access to a larger number of investment opportunities', 'To invest in mutual funds set up by the network', 'To leverage the experience of other network members', 'To benefit from educational or training sessions', 'To expand my personal and professional network', 'To coinvest with other investors', 'Other',],
    #   ['More opportunities', 'Network funds', "Members' experience", 'Training', 'Expand network', 'Coinvest', 'Other',],
    # ], # Only BA  
    # ['inv_num', 'Number of Investments', ['None', '1-2', '3-5', '6-10', '11-20', '>20'],],
    # ['inv_companies_last2y', 'Last 2Y Investments'], # Text
    # ['inv_amount_average', 'Average Invested Amount',
    #    [
    #     ['<25K€', '25k€-50k€', '50k€-100k€', '100k€-500k€', '>500k€', ],
    #     ['<0.5K€', '0.5k€-1k€', '1k€-5k€', '5k€-10k€', '10k€-50k€', '>50k€'],
    # ],], # Not same labels
    # ['inv_capital_stake', 'Typical Stake', ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%', '>51%'],], # Only for BA
    # ['inv_dd_duration', 'Analysis Duration',  [
    #     ['Less than 1 month', '1-6 months', '7-12 months', '13-18 months', 'More than 18 months'],
    #     ['Less than 1 hour', '1 to 2 hours', '3 to 5 hours', '6 to 10 hours', 'More than 10 hours'],
    # ],], # Not same labels
    # ['inv_criteria', 'Investing Criteria',  [
    #     # ['Alignment with my experience and expertise', 'Business plan', 'Scalability/Market potential', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    #     # ['Business plan', 'Crowd opinion', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    #     ['Expertise', 'Business plan', 'Scalability', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    #     ['Business plan', 'Crowd opinion', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    # ],], # Not same labels multiple
    # ['inv_sectors', 'Sectors', 
    #     # ['Information technology (software, internet, IT services, etc.)', 'Finance', 'Healthcare / Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    #     ['IT', 'Finance', 'Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    # ],
    # ['inv_process', 'Investing Process',
    #     # ['I invest alone (as an individual investor)', 'I invest as part of an angel syndicate or group', 'I invest through venture capital funds', 'I invest through crowdfunding platforms', 'Other'],
    #     ['Alone', 'Syndicate/group', 'Venture capital', 'Crowdfunding platforms', 'Other'],
    # ], # Only BA
    # ['inv_visits', 'Number of Visits', ['0 per year', '1-2 per year', '3-6 per year', '7-12 per year', 'More than 12 per year'],], # Only BA
    # ['non_fin_contr', 'Non-Financial Contribution',
    #     # ['Strategic guidance', 'Networking opportunities', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring and accountability', 'Other',],
    #     ['Strategy', 'Networking', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring', 'Other',],
    # ], # Only BA
    # ['child_bias', 'Children Effect', ['1 - Not at all', '2', '3', '4', '5 - Very much',],], # Only BA
    # ['inv_goal', 'Investment Goal',
    #     # ['Capital preservation (low risk, low return)', 'Income generation (moderate risk, moderate return)', 'Growth (high risk, high return)', 'Speculative (very high risk, very high return)'],
    #     ['Capital preservation', 'Income generation', 'Growth', 'Speculative'],
    # ], 
    # ['inv_exit', 'Exit', [
    #     # ['No, I have not sold my investments yet', 'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
    #     # ['No, I have not sold my investments yet', "Yes, through the platform’s secondary market",  'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
    #     ['Not sold yet', 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
    #     ['Not sold yet', "Platform’s secondary market", 'Private transaction', 'Company acquisition', 'IPO', 'Company buyback', 'Cessation'], 
    # ] ], # Not same labels
    # ['inv_return', 'Investment Returns',
    #     # ['I have no idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    #     ['No idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    # ],
    # ['women_investing', 'Women Investing Knowledge', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    # ['women_involvement', 'Women Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']], 
    # ['women_influence', 'Women Influence', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_diff_eval_process', 'Women Evaluation Process', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_risk_averse', 'Women Risk Aversion', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_evolution_involvement', 'Women Evolution Involvement', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['women_evolution_neg_stereotype', 'Women Stereotype Evolution', ['Strongly disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly agree']],
    # ['mother_needs_consideration', "Consideration of Mothers' Needs", ['1-Not at all', '2', '3', '4', '5-Completely',]],
    # ['women_reco', 'Recommendations for Attracting Women',[
    #     # ['Increase visibility of successful women business angels', 'Offer targeted networking events for women', 'Highlight women-led projects', 'Offer training or mentorship programs for aspiring women investors', 'Establish women-focused investment funds or syndicates', 'Other'],
    #     # ['Create events or online communities where women can network', 'Ensure the platform is user-friendly and inclusive', 'Highlight women-led projects', 'Offer training programs', 'Showcase successful women crowd-investors in the investment space', 'Other']
    #     ['Communication on success', 'Networking', 'Communication on projects', 'Training', 'Specific funds', 'Other'],
    #     ['Networking', 'User-Friendliness','Communication on projects', 'Training', 'Communication on success', 'Other'],
    # ]],
    # ['women_led_inv', 'Number of Women-Led Investments', ['None', '1-2', '3-5', '6-10', '>10']],
]

categories=[
    'Investor Experience', 
    # 'Motivations',
    'Part of BA Network',
    # 'Investment Platform',
    # 'Part of BA Network Motivations',
    'Number of Investments', 
    # 'Last 2Y Investments',
    # 'Average Invested Amount', 
    'Typical Stake',
    # 'Analysis Duration',
    # 'Investing Criteria',
    # 'Sectors',
    # 'Investing Process',
    'Number of Visits',
    # 'Non-Financial Contribution',
    'Children Effect',
    'Investment Goal', 
    # 'Exit',
    'Returns',
    ]

category_orders=[
    # ['Within the last year','2-3 years ago', '4-6 years ago', '7-9 years ago', '10 or more years ago'],
    ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],
    # [
    #     ['Support innovative startups or entrepreneurs', 'Potential high returns on investments', 'To diversify my investment portfolio', 'Recommendation from friends', 'Benefits from tax breaks', 'My financial capacity', 'My family background', 'Other' ],
    #     ['Desire to support innovative startups or entrepreneurs', 'Potential high returns on investments', 'Diversifying my investment portfolio', 'Recommendation from friends or media influence', 'Benefits from tax breaks', 'Business reasons (you could be a supplier, etc.)', 'Other' ],
    # ],
    # [
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Financial capacity', 'Family background', 'Other' ],
    #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Business reasons', 'Other' ],
    # ],
    # ['Yes', 'No'],
    # ['To gain access to a larger number of investment opportunities', 'To invest in mutual funds set up by the network', 'To leverage the experience of other network members', 'To benefit from educational or training sessions', 'To expand my personal and professional network', 'To coinvest with other investors', 'Other',]
    # ['More opportunities', 'Network funds', "Members' experience", 'Training', 'Expand network', 'Coinvest', 'Other',]
    # ['None', '1-2', '3-5', '6-10', '11-20', 'More than 20'],
    # ['None', '1-2', '3-5', '6-10', '11-20', '>20'],
    # [
    #     ['<25K€', '25k€-50k€', '50k€-100k€', '100k€-500k€', '>500k€', ],
    #     ['<0.5K€', '0.5k€-1k€', '1k€-5k€', '5k€-10k€', '10k€-50k€', '>50k€'],
    # ],
    # ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%', '>51%'],
    # ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%'],
    # [
    #     ['Less than 1 month', '1-6 months', '7-12 months', '13-18 months', 'More than 18 months'],
    #     ['Less than 1 hour', '1 to 2 hours', '3 to 5 hours', '6 to 10 hours', 'More than 10 hours'],
    # ],
    # [
    #     ['Alignment with my experience and expertise', 'Business plan', 'Scalability/Market potential', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    #     ['Business plan', 'Crowd opinion', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
    # ],
    # [
    #     ['Expertise', 'Business plan', 'Scalability', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    #     ['Business plan', 'Crowd opinion', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
    # ],
    # ['Information technology (software, internet, IT services, etc.)', 'Finance', 'Healthcare / Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    # ['IT', 'Finance', 'Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
    # ['I invest alone (as an individual investor)', 'I invest as part of an angel syndicate or group', 'I invest through venture capital funds', 'I invest through crowdfunding platforms', 'Other'],
    # ['Alone', 'Syndicate/group', 'Venture capital', 'Crowdfunding platforms', 'Other'],
    # ['0 per year', '1-2 per year', '3-6 per year', '7-12 per year', 'More than 12 per year'],
    # ['Strategic guidance', 'Networking opportunities', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring and accountability', 'Other',],
    # ['Strategy', 'Networking', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring', 'Other',],
    # ['1- Not at all', '2', '3', '4', '5 - Very much',],
    # ['Capital preservation (low risk, low return)', 'Income generation (moderate risk, moderate return)', 'Growth (high risk, high return)', 'Speculative (very high risk, very high return)'],
    # ['Capital preservation', 'Income generation', 'Growth', 'Speculative'],
    # ['No, I have not sold my investments yet', 'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
    # ['I have no idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
    # ['No idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],


]
    
# for index in range(len(columns)):
#     main_title=columns[index][1]
#     x_title=''
#     x_label=columns[index][0]
#     y_title='Frequency'

#     hist(df, main_title, x_title, x_label, y_title, columns[index][2])
hist(df, columns)


Investor Experience - Gender


Investor Experience - Investor Type


Investor Experience - Gender & Investor Type


In [104]:
df_load['ba_group_reason'].head()

0    3,5,6
1    1,3,5
2    1,3,6
3    1,3,6
4    1,3,6
Name: ba_group_reason, dtype: object

In [ ]:
# Distribution for multiple choice but same list of options
import pandas as pd 


df=pd.DataFrame()

def multiple_choice(row, df=df):
    column='ba_group_reason'
    # column='inv_sectors'
    # print(row[column])
    if not (pd.isna(row[column])):    
        if isinstance(row[column], str):
            options=row[column].split(',')
        else:
            options=[row[column]]
    # print(options)
    
    for option in options:
        new_row=row.copy()
        new_row[column]=option
        print(new_row[column])
        df=pd.concat([df, new_row])
        

df_load.iloc[:5,:].apply(multiple_choice, axis=1)



display(df)

# columns=[
#     # 'inv_time',
#     # 'inv_motivation', # Multiple not the same labels
#     # 'ba_group', # Boolean
#     # 'ba_group_name', # Not same labels
#     'ba_group_reason', # Multiple
#     # 'inv_num',
#     # 'inv_companies_last2y', # Text
#     # 'inv_amount_average', # Not same labels
#     # 'inv_capital_stake',
#     # 'inv_dd_duration', # Not same labels
#     # 'inv_criteria', # Multiple Not same labels
#     'inv_sectors', # Multiple
#     # 'inv_process',
#     # 'inv_visits',
#     # 'non_fin_contr', # Not the same labels
#     # 'child_bias', 
#     # 'inv_goal',
#     # 'inv_exit', # Multiple, Not the same labels
#     # 'inv_return',

# ]

# categories=[
#     # 'Investor Experience', 
#     # 'Motivations',
#     # 'Part of BA Network',
#     # 'Investment Platform',
#     'Part of BA Network Motivations',
#     # 'Number of Investments', 
#     # 'Last 2Y Investments',
#     # 'Average Invested Amount', 
#     # 'Typical Stake',
#     # 'Analysis Duration',
#     # 'Investing Criteria',
#     'Sectors',
#     # 'Investing Process',
#     # 'Number of Visits',
#     # 'Non-Financial Contribution',
#     # 'Children Effect',
#     # 'Investment Goal', 
#     # 'Exit',
#     # 'Returns',
#     ]

# category_orders=[
#     # ['Within the last year','2-3 years ago', '4-6 years ago', '7-9 years ago', '10 or more years ago'],
#     # ['<1 year','2-3 years', '4-6 years', '7-9 years', '> 10 years'],
#     # [
#     #     ['Support innovative startups or entrepreneurs', 'Potential high returns on investments', 'To diversify my investment portfolio', 'Recommendation from friends', 'Benefits from tax breaks', 'My financial capacity', 'My family background', 'Other' ],
#     #     ['Desire to support innovative startups or entrepreneurs', 'Potential high returns on investments', 'Diversifying my investment portfolio', 'Recommendation from friends or media influence', 'Benefits from tax breaks', 'Business reasons (you could be a supplier, etc.)', 'Other' ],
#     # ],
#     # [
#     #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Financial capacity', 'Family background', 'Other' ],
#     #     ['Support entrepreneurs', 'High returns', 'Diversify portfolio', 'Friend recommendation', 'Tax breaks', 'Business reasons', 'Other' ],
#     # ],
#     # ['Yes', 'No'],
#     # ['To gain access to a larger number of investment opportunities', 'To invest in mutual funds set up by the network', 'To leverage the experience of other network members', 'To benefit from educational or training sessions', 'To expand my personal and professional network', 'To coinvest with other investors', 'Other',]
#     ['More opportunities', 'Network funds', "Members' experience", 'Training', 'Expand network', 'Coinvest', 'Other',]
#     # ['None', '1-2', '3-5', '6-10', '11-20', 'More than 20'],
#     # ['None', '1-2', '3-5', '6-10', '11-20', '>20'],
#     # [
#     #     ['<25K€', '25k€-50k€', '50k€-100k€', '100k€-500k€', '>500k€', ],
#     #     ['<0.5K€', '0.5k€-1k€', '1k€-5k€', '5k€-10k€', '10k€-50k€', '>50k€'],
#     # ],
#     # ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%', '>51%'],
#     # ['0-5%', '6-10%', '11-20%', '21-30%', '31-50%'],
#     # [
#     #     ['Less than 1 month', '1-6 months', '7-12 months', '13-18 months', 'More than 18 months'],
#     #     ['Less than 1 hour', '1 to 2 hours', '3 to 5 hours', '6 to 10 hours', 'More than 10 hours'],
#     # ],
#     # [
#     #     ['Alignment with my experience and expertise', 'Business plan', 'Scalability/Market potential', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
#     #     ['Business plan', 'Crowd opinion', 'Entrepreneurial team', 'Environmental/social impact', 'Exit strategy', 'Fair valuation/return', 'Other'],
#     # ],
#     # [
#     #     ['Expertise', 'Business plan', 'Scalability', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
#     #     ['Business plan', 'Crowd opinion', 'Team', 'ES impact', 'Exit', 'Return', 'Other'],
#     # ],
#     # ['Information technology (software, internet, IT services, etc.)', 'Finance', 'Healthcare / Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
#     ['IT', 'Finance', 'Biotech', 'Manufacturing', 'Retail', 'Consulting', 'Energy', 'Transportation', 'Other'],
#     # ['I invest alone (as an individual investor)', 'I invest as part of an angel syndicate or group', 'I invest through venture capital funds', 'I invest through crowdfunding platforms', 'Other'],
#     # ['Alone', 'Syndicate/group', 'Venture capital', 'Crowdfunding platforms', 'Other'],
#     # ['0 per year', '1-2 per year', '3-6 per year', '7-12 per year', 'More than 12 per year'],
#     # ['Strategic guidance', 'Networking opportunities', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring and accountability', 'Other',],
#     # ['Strategy', 'Networking', 'Financial and legal expertise', 'Technical or industrial knowledge', 'Monitoring', 'Other',],
#     # ['1- Not at all', '2', '3', '4', '5 - Very much',],
#     # ['Capital preservation (low risk, low return)', 'Income generation (moderate risk, moderate return)', 'Growth (high risk, high return)', 'Speculative (very high risk, very high return)'],
#     # ['Capital preservation', 'Income generation', 'Growth', 'Speculative'],
#     # ['No, I have not sold my investments yet', 'Yes, through a private transaction', 'Yes, through the company acquisition', 'Yes, through an initial public offering', 'Yes, through a company buyback program', 'Yes, through cessation of the company activity'], 
#     # ['I have no idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],
#     # ['No idea', 'Highly negative', 'Negative', 'Neutral', 'Positive', 'Highly positive'],


# ]
    
# for index in range(len(columns)):
#     main_title=categories[index]
#     x_title=''
#     x_label=columns[index]
#     y_title='Frequency'

#     hist_unique_cross(df, main_title, x_title, x_label, y_title, category_orders[index])


1
2
1
1
3
9
1
3
5
3
4
9


""


In [24]:
df_load[['inv_criteria', 'inv_sectors']].head()

,inv_criteria,inv_sectors
0,"3,4","1,2"
1,"1,3,4",1
2,"3,4,5","1,3,9"
3,"2,3,4","1,3,5"
4,"2,3,4","3,4,9"


# Investment Criteria

## Business Angels

Q3.11 What are the most important factors you consider before making an investment? (max 3 options)
▢	Alignment with my experience and expertise  (1) 
▢	Business plan  (2) 
▢	Scalability/Market potential  (3) 
▢	Entrepreneurial team  (4) 
▢	Environmental/social impact  (5) 
▢	Exit strategy  (6) 
▢	Fair valuation/return  (7) 
▢	Other (to clarify)  (8) __________________________________________________

## Crowfunding Investors

Q3.8 What are the main factors you consider before making an investment via a crowdfunding platform? (max 3 options)
▢	Business plan  (1) 
▢	Crowd opinion  (2) 
▢	Entrepreneurial team  (3) 
▢	Environmental/social impact  (4) 
▢	Exit strategy  (5) 
▢	Fair valuation/return  (6) 
▢	Other (to clarify)  (7) __________________________________________________

## Consolidated questionaire

1. Business plan
2. Entrepreneurial team
3. Environmental/social impact 
4. Exit strategy
5. Fair valuation/return 
6. Alignment with my experience and expertise
7. Scalability/Market potential
8. Crowd opinion
9. Other


,gender_male,mode,median,mean,std,count,var
0,1.0,3.0,3.0,3.064516,1.093480,31,inv_time
1,2.0,5.0,3.0,3.372093,1.337658,86,inv_time


In [ ]:
# Statistics - Gender
funcs=[mode, 'median', 'mean', 'std', 'count']
groups=['gender_male']
columns=['inv_time',]

basic_stats(funcs, groups, columns)

In [ ]:
# inv_sectors

# BA
# Q3.12 In which business sector(s) have you invested as a business angel?
# ▢	Information technology (software, internet, IT services, etc.)  (1) 
# ▢	Finance  (2) 
# ▢	Healthcare / Biotech  (3) 
# ▢	Manufacturing  (4) 
# ▢	Retail  (5) 
# ▢	Consulting  (6) 
# ▢	Energy  (7) 
# ▢	Transportation  (8) 
# ▢	Other (please clarify)  (9) __________________________________________________

# CI
# Q3.9 In which business sector(s) have you invested as a crowd investor?
# ▢	Information technology (software, internet, IT services, etc.)  (1) 
# ▢	Finance  (2) 
# ▢	Healthcare / Biotech  (3) 
# ▢	Manufacturing  (4) 
# ▢	Retail  (5) 
# ▢	Consulting  (6) 
# ▢	Energy  (7) 
# ▢	Transportation  (8) 
# ▢	Other (please clarify)  (9) __________________________________________________
